# IBMDS_Capstone_C3

In [39]:
import pandas as pd
import numpy as np
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe


In [5]:
import lxml

link = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

df = pd.read_html(link,header=0)[0]

In [6]:
df.shape

(287, 3)

#Step 1: filter not assigned

In [7]:
# Only process the cells that have an assigned borough.
df_filtered = df[df['Borough'] != 'Not assigned'] 

In [8]:
df_filtered.shape

(210, 3)

#STEP 2 <br>
More than one neighborhood can exist in one postal code area.
For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has 
two neighborhoods: Harbourfront and Regent Park. 
These two rows will be combined into one row with the neighborhoods 
separated with a comma as shown in row 11 in the above table.

In [9]:
df3=df_filtered.groupby(['Postcode','Borough'])['Neighbourhood'].apply(','.join).reset_index()

In [10]:
#pd.set_option('display.max_rows', df.shape[0]+1)
#print(df3)

#Step 3 <br>
replace not assigned neighbourhood with borough name

In [11]:

df3.loc[df3["Neighbourhood"] == 'Not assigned','Neighbourhood'] = df3["Borough"]

#PART II

In [12]:
geo=pd.read_csv('./Geospatial_Coordinates.csv')

In [13]:
#geogeo.set_index('Postal Code', inplace=True)

In [14]:
#df = pd.merge(df3, geo, on=['Postcode','Postal Code'])
df_merge_difkey = pd.merge(df3, geo, left_on='Postcode', right_on='Postal Code')

In [29]:
df_Toronto

,Postcode,Borough,Neighbourhood,Postal Code,Latitude,Longitude
37,M4E,East Toronto,The Beaches,M4E,43.676357,-79.293031
41,M4K,East Toronto,"The Danforth West,Riverdale",M4K,43.679557,-79.352188
42,M4L,East Toronto,"The Beaches West,India Bazaar",M4L,43.668999,-79.315572
43,M4M,East Toronto,Studio District,M4M,43.659526,-79.340923
44,M4N,Central Toronto,Lawrence Park,M4N,43.728020,-79.388790
45,M4P,Central Toronto,Davisville North,M4P,43.712751,-79.390197
46,M4R,Central Toronto,North Toronto West,M4R,43.715383,-79.405678
47,M4S,Central Toronto,Davisville,M4S,43.704324,-79.388790
48,M4T,Central Toronto,"Moore Park,Summerhill East",M4T,43.689574,-79.383160
49,M4V,Central Toronto,"Deer Park,Forest Hill SE,Rathnelly,South Hill,...",M4V,43.686412,-79.400049


#PART III

In [24]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

Explore and cluster the neighborhoods in Toronto. You can decide to work with only boroughs that contain the word Toronto and then replicate the same analysis we did to the New York City data. It is up to you.
to add enough Markdown cells to explain what you decided to do and to report any observations you make.
to generate maps to visualize your neighborhoods and how they cluster together.

In [132]:
df_Toronto = df_merge_difkey[df_merge_difkey['Borough'].str.contains('Toronto', na=False)].reset_index(drop=True)
df_Toronto.rename(columns={"Neighbourhood": "Neighborhood"},inplace=True)

In [134]:
df_Toronto.shape

(39, 6)

In [34]:
# create map of New York using latitude and longitude values
[latitude, longitude]=[43.6532,-79.3832]
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_Toronto['Latitude'], df_Toronto['Longitude'], df_Toronto['Borough'], df_Toronto['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto 

In [35]:
CLIENT_ID = 'NWKXNY1DDWEHWQRRHVHJVGC2CG50R2JPI4SFV2H3WOZTWVUJ' # your Foursquare ID
CLIENT_SECRET = 'LOPFOWE3EEHUCAISBA5VST0V0ZYYJRDFMKKPZFZ5A02VZMUN' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: NWKXNY1DDWEHWQRRHVHJVGC2CG50R2JPI4SFV2H3WOZTWVUJ
CLIENT_SECRET:LOPFOWE3EEHUCAISBA5VST0V0ZYYJRDFMKKPZFZ5A02VZMUN


In [152]:
def getNearbyVenues(names, latitudes, longitudes, radius=250):
    LIMIT = 100 
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [153]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [155]:

toronto_venues = getNearbyVenues(names=df_Toronto['Neighborhood'],
                                   latitudes=df_Toronto['Latitude'],
                                   longitudes=df_Toronto['Longitude']
                                  )

The Beaches
The Danforth West,Riverdale
The Beaches West,India Bazaar
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park,Summerhill East
Deer Park,Forest Hill SE,Rathnelly,South Hill,Summerhill West
Rosedale
Cabbagetown,St. James Town
Church and Wellesley
Harbourfront
Ryerson,Garden District
St. James Town
Berczy Park
Central Bay Street
Adelaide,King,Richmond
Harbourfront East,Toronto Islands,Union Station
Design Exchange,Toronto Dominion Centre
Commerce Court,Victoria Hotel
Roselawn
Forest Hill North,Forest Hill West
The Annex,North Midtown,Yorkville
Harbord,University of Toronto
Chinatown,Grange Park,Kensington Market
CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara
Stn A PO Boxes 25 The Esplanade
First Canadian Place,Underground city
Christie
Dovercourt Village,Dufferin
Little Portugal,Trinity
Brockton,Exhibition Place,Parkdale Village
High Park,The Junction South
Parkdale,Roncesvalles
Runnymede

In [156]:
print(toronto_venues.shape)
toronto_venues.head()

(652, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,The Beaches,43.676357,-79.293031,Glen Stewart Park,43.675278,-79.294647,Park
2,The Beaches,43.676357,-79.293031,Glen Stewart Ravine,43.676300,-79.294784,Other Great Outdoors
3,"The Danforth West,Riverdale",43.679557,-79.352188,MenEssentials,43.677820,-79.351265,Cosmetics Shop
4,"The Danforth West,Riverdale",43.679557,-79.352188,Pantheon,43.677621,-79.351434,Greek Restaurant


In [157]:
## 2. Explore Neighborhoods in Manhattan

In [158]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide,King,Richmond",34,34,34,34,34,34
Berczy Park,3,3,3,3,3,3
"Brockton,Exhibition Place,Parkdale Village",9,9,9,9,9,9
Business Reply Mail Processing Centre 969 Eastern,3,3,3,3,3,3
"CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara",2,2,2,2,2,2
"Cabbagetown,St. James Town",16,16,16,16,16,16
Central Bay Street,22,22,22,22,22,22
"Chinatown,Grange Park,Kensington Market",44,44,44,44,44,44
Christie,7,7,7,7,7,7


In [159]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 159 uniques categories.


In [160]:
## 3. Analyze Each Neighborhood

In [161]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Accessories Store,Adult Boutique,Airport Lounge,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,...,Theater,Theme Restaurant,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [162]:
toronto_onehot.shape

(652, 159)

In [163]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Accessories Store,Adult Boutique,Airport Lounge,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,...,Theater,Theme Restaurant,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint
0,"Adelaide,King,Richmond",0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.058824,0.0,...,0.000000,0.000000,0.000000,0.000000,0.029412,0.000000,0.000000,0.000000,0.000000,0.000000
1,Berczy Park,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,"Brockton,Exhibition Place,Parkdale Village",0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.111111,0.000000,0.000000,0.000000
3,Business Reply Mail Processing Centre 969 Eastern,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,"CN Tower,Bathurst Quay,Island airport,Harbourf...",0.000000,0.0,0.000000,0.5,0.000000,0.000000,0.000000,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,"Cabbagetown,St. James Town",0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,Central Bay Street,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,"Chinatown,Grange Park,Kensington Market",0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.022727,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.045455,0.000000,0.068182,0.022727,0.000000,0.000000
8,Christie,0.000000,0.0,0.000000,0.0,0.142857,0.000000,0.000000,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,Church and Wellesley,0.000000,0.0,0.028571,0.0,0.000000,0.000000,0.000000,0.000000,0.0,...,0.000000,0.028571,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.028571,0.028571


In [164]:
toronto_grouped.shape

(38, 159)

In [165]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide,King,Richmond----
                venue  freq
0          Steakhouse  0.09
1         Coffee Shop  0.06
2  Seafood Restaurant  0.06
3                 Bar  0.06
4    Asian Restaurant  0.06


----Berczy Park----
                venue  freq
0      Breakfast Spot  0.33
1          Steakhouse  0.33
2        Liquor Store  0.33
3         Yoga Studio  0.00
4  Light Rail Station  0.00


----Brockton,Exhibition Place,Parkdale Village----
                   venue  freq
0                   Café  0.22
1                    Pub  0.11
2      Convenience Store  0.11
3  Vietnamese Restaurant  0.11
4         Sandwich Place  0.11


----Business Reply Mail Processing Centre 969 Eastern----
           venue  freq
0         Garden  0.33
1  Auto Workshop  0.33
2        Brewery  0.33
3    Yoga Studio  0.00
4  Movie Theater  0.00


----CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara----
                   venue  freq
0  Performing Arts Venue   0.5
1

In [166]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [194]:
## 4. Cluster Neighborhoods

In [209]:
# set number of clusters
kclusters = 8

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:] 

array([0, 0, 0, 5, 6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 2,
       0, 0, 4, 5, 0, 0, 7, 0, 0, 0, 0, 0, 3, 0, 0, 0], dtype=int32)

In [210]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide,King,Richmond",Steakhouse,Bar,Coffee Shop,Hotel,Seafood Restaurant,Asian Restaurant,Park,Greek Restaurant,Opera House,Gym
1,Berczy Park,Breakfast Spot,Steakhouse,Liquor Store,Wings Joint,Dessert Shop,Electronics Store,Dumpling Restaurant,Donut Shop,Discount Store,Diner
2,"Brockton,Exhibition Place,Parkdale Village",Café,Japanese Restaurant,Vietnamese Restaurant,Sandwich Place,Convenience Store,Restaurant,Pizza Place,Pub,Wings Joint,Discount Store
3,Business Reply Mail Processing Centre 969 Eastern,Auto Workshop,Brewery,Garden,Department Store,Ethiopian Restaurant,Electronics Store,Dumpling Restaurant,Donut Shop,Discount Store,Diner
4,"CN Tower,Bathurst Quay,Island airport,Harbourf...",Performing Arts Venue,Airport Lounge,Deli / Bodega,Electronics Store,Dumpling Restaurant,Donut Shop,Discount Store,Diner,Dessert Shop,Department Store


In [211]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

neighborhoods_venues_sorted.head()


,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,0,"Adelaide,King,Richmond",Steakhouse,Bar,Coffee Shop,Hotel,Seafood Restaurant,Asian Restaurant,Park,Greek Restaurant,Opera House,Gym
1,0,Berczy Park,Breakfast Spot,Steakhouse,Liquor Store,Wings Joint,Dessert Shop,Electronics Store,Dumpling Restaurant,Donut Shop,Discount Store,Diner
2,0,"Brockton,Exhibition Place,Parkdale Village",Café,Japanese Restaurant,Vietnamese Restaurant,Sandwich Place,Convenience Store,Restaurant,Pizza Place,Pub,Wings Joint,Discount Store
3,5,Business Reply Mail Processing Centre 969 Eastern,Auto Workshop,Brewery,Garden,Department Store,Ethiopian Restaurant,Electronics Store,Dumpling Restaurant,Donut Shop,Discount Store,Diner
4,6,"CN Tower,Bathurst Quay,Island airport,Harbourf...",Performing Arts Venue,Airport Lounge,Deli / Bodega,Electronics Store,Dumpling Restaurant,Donut Shop,Discount Store,Diner,Dessert Shop,Department Store


In [212]:
Toronto_merged = df_Toronto

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
Toronto_merged = Toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
Toronto_merged['Cluster Labels'] = Toronto_merged['Cluster Labels'].astype(int)
Toronto_merged.head() # check the last columns!

ValueError: Cannot convert non-finite values (NA or inf) to integer

In [201]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Toronto_merged['Latitude'], Toronto_merged['Longitude'], Toronto_merged['Neighborhood'], Toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    print(rainbow,cluster)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

['#8000ff', '#386df9', '#12c8e6', '#5af8c8', '#a4f89f', '#ecc86f', '#ff6d38', '#ff0000'] 0.0


TypeError: list indices must be integers or slices, not float

In [119]:
## 5. Examine Clusters

In [142]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 0, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]




,Borough,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,East Toronto,-79.352188,0,Greek Restaurant,Coffee Shop,Italian Restaurant,Restaurant,Ice Cream Shop,Bookstore,Furniture / Home Store,Grocery Store,Frozen Yogurt Shop,Fruit & Vegetable Store
3,East Toronto,-79.340923,0,Café,Coffee Shop,Bakery,Gastropub,Italian Restaurant,American Restaurant,Brewery,Bar,Ice Cream Shop,Comfort Food Restaurant
6,Central Toronto,-79.405678,0,Clothing Store,Coffee Shop,Salon / Barbershop,Restaurant,Café,Chinese Restaurant,Pet Store,Park,Cosmetics Shop,Sporting Goods Shop
7,Central Toronto,-79.388790,0,Sandwich Place,Pizza Place,Dessert Shop,Italian Restaurant,Coffee Shop,Café,Gym,Sushi Restaurant,Farmers Market,Indian Restaurant
9,Central Toronto,-79.400049,0,Pub,Coffee Shop,Liquor Store,Supermarket,Sushi Restaurant,Sports Bar,Pizza Place,Restaurant,Fried Chicken Joint,American Restaurant
11,Downtown Toronto,-79.367675,0,Coffee Shop,Pub,Park,Pizza Place,Café,Bakery,Convenience Store,Restaurant,Italian Restaurant,Butcher
12,Downtown Toronto,-79.383160,0,Coffee Shop,Japanese Restaurant,Gay Bar,Sushi Restaurant,Restaurant,Gym,Mediterranean Restaurant,Café,Hotel,Burger Joint
13,Downtown Toronto,-79.360636,0,Coffee Shop,Pub,Park,Café,Bakery,Mexican Restaurant,Restaurant,Breakfast Spot,Event Space,Hotel
14,Downtown Toronto,-79.378937,0,Coffee Shop,Clothing Store,Japanese Restaurant,Cosmetics Shop,Café,Middle Eastern Restaurant,Diner,Ramen Restaurant,Bookstore,Bakery
15,Downtown Toronto,-79.375418,0,Coffee Shop,Café,Restaurant,Breakfast Spot,Clothing Store,American Restaurant,Bakery,Cocktail Bar,Cosmetics Shop,Italian Restaurant


In [143]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 1, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]




,Borough,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,Central Toronto,-79.383160,1,Park,Playground,Trail,Tennis Court,Cuban Restaurant,Donut Shop,Doner Restaurant,Dog Run,Discount Store,Diner
10,Downtown Toronto,-79.377529,1,Park,Playground,Trail,Cuban Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run,Discount Store,Diner


In [144]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 2, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]




,Borough,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
22,Central Toronto,-79.416936,2,Garden,Music Venue,Women's Store,Dance Studio,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run,Discount Store


In [145]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 3, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]




,Borough,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
23,Central Toronto,-79.411307,3,Jewelry Store,Trail,Mexican Restaurant,Sushi Restaurant,Dance Studio,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run,Discount Store


In [146]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 4, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]




,Borough,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
38,East Toronto,-79.321558,4,Light Rail Station,Burrito Place,Restaurant,Skate Park,Garden Center,Garden,Brewery,Park,Comic Shop,Smoke Shop


In [147]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 5, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]




,Borough,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Central Toronto,-79.390197,5,Park,Breakfast Spot,Gym,Sandwich Place,Department Store,Food & Drink Shop,Hotel,Dance Studio,Donut Shop,Doner Restaurant


In [148]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 6, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]




,Borough,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Central Toronto,-79.38879,6,Park,Construction & Landscaping,Bus Line,Swim School,Dance Studio,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run,Discount Store


In [149]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 7, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]




,Borough,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,East Toronto,-79.293031,7,Trail,Health Food Store,Pub,Women's Store,Cupcake Shop,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run,Discount Store


In [150]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 8, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]




,Borough,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,East Toronto,-79.315572,8,Sandwich Place,Park,Sushi Restaurant,Burger Joint,Pub,Fast Food Restaurant,Ice Cream Shop,Food & Drink Shop,Liquor Store,Burrito Place


In [151]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 9, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]




,Borough,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
35,West Toronto,-79.456325,9,Gift Shop,Breakfast Spot,Movie Theater,Eastern European Restaurant,Italian Restaurant,Dog Run,Bar,Dessert Shop,Restaurant,Cuban Restaurant
